In [ ]:
import pandas as pd
from src.swinno_helpers import split_cols
from src.swinno_helpers import melt_table


In [ ]:
tags_221222 = pd.read_csv("../data/raw-data/221222-tagged-innovations.csv")
tags_221222.columns = tags_221222.columns.str.strip().str.lower().str.replace(" ", "_")
tags_221222.head()


## Check for Duplicates

In [ ]:
duplicates = tags_221222.loc[tags_221222.duplicated(subset="sinno_id", keep=False), :]
duplicates = duplicates.sort_values(by="sinno_id")
len(duplicates)


There are 46 duplicates in this dataset.
The majority is non forest.

While some innovations have received at least one differing code, the overwhelming majority has been tagged as close enough.

There is also a possibility that the introduction of some codes later in the process influenced the categorization.
Need to check this up

## Prepare DB Entry

In [ ]:
innovation_types = tags_221222[["sinno_id", "innovation_type"]]
visions = tags_221222[["sinno_id", "bioeconomy_vision"]]
notes = tags_221222[["sinno_id", "notes"]]


### Innovation Types

In [ ]:
split_innovation_types = split_cols(innovation_types, "innovation_type", sep=",")


In [ ]:
melted_innovation_types = melt_table(
    split_innovation_types, "sinno_id", "innovation", "innovation_type"
).dropna()


In [ ]:
melted_innovation_types.head()


In [ ]:
melted_innovation_types["sinno_id"] = melted_innovation_types["sinno_id"].astype(int)


### Bioeconomy Visions

In [ ]:
visions.columns


In [ ]:
split_visions = split_cols(visions, col_to_split="bioeconomy_vision", sep=",")
melted_visions = melt_table(
    split_visions, id_vars="sinno_id", col_start="bio", value_name="bioeconomy_vision"
)
melted_visions.dropna(inplace=True)
melted_visions["sinno_id"] = melted_visions["sinno_id"].astype(int)


## Export


In [ ]:
notes.to_csv("../data/modified-data/22122-notes.csv")
melted_visions.to_csv("../data/modified-data/221222-visions.csv")
melted_innovation_types.to_csv("../data/modified-data/221222-innovation-types.csv")


In [ ]:
from pathlib import Path
from sqlalchemy import create_engine
database_dir = Path.cwd().parent.parent
database_uri = f'sqlite:///{database_dir}/swinno.db'


In [ ]:
engine = create_engine(database_uri)

In [12]:
from src.swinno_helpers import connect_swinno_db

connect_swinno_db()

NameError: name 'swinno' is not defined

In [ ]:
melted_innovation_types.to_sql(name='eco_innovations', con=engine, if_exists='append', index=False)


In [ ]:
notes.to_sql(name='classification_notes', con=engine, if_exists='append', index=False)

In [ ]:
melted_visions.to_sql(name='bioeconomy_vision', con=engine, if_exists='append', index=False)